### Check if miners get reward correctly

In [2]:
import requests
import json
import torch
from dependency_modules.rewarding.app import instantiate_from_config, MODEL_CONFIG
from image_generation_subnet.validator.forward import get_reward
from image_generation_subnet.protocol import ImageGenerating
from dependency_modules.rewarding.utils import pil_image_to_base64
import numpy as np

N = 20
url_dict = {
    "SDXLTurbo": "http://sdxl_turbo_nicheimage.nichetensor.com:15012/verify",
    "RealisticVision": "http://check_realistic_vision_nicheimage.nichetensor.com:15011/verify",
}
rewards = {}
for model, url in url_dict.items():
    print(f"Model: {model}")
    pipe = instantiate_from_config(MODEL_CONFIG[model])
    rewards[model] = []
    if model == "RealisticVision":
        params = {
            "num_inference_steps": 30,
        }
    else:
        params = {
            "num_inference_steps": 4,
            "width": 512,
            "height": 512,
            "guidance_scale": 0.5,
        }
    for i in range(N):
        prompt = "an image of samurai, red hat, and a katana, with a red background"
        seed = i
        synapse = ImageGenerating(
            prompt=prompt,
            seed=seed,
            model_name=model,
            pipeline_params=params,
        )
        generator = torch.manual_seed(seed)
        result = pipe(
            prompt,
            generator=generator,
            **params,
        )
        images = result.images
        synapse.image = pil_image_to_base64(images[0])
        reward = get_reward(url, [synapse], synapse)[0]
        rewards[model].append(reward)

    print(f"Model: {model}, rewards: {rewards[model]}")
    print(f"Accuracy: {np.mean(rewards[model])}")

Model: SDXLTurbo


100%|██████████| 4/4 [00:00<00:00, 15.30it/s]


Model: SDXLTurbo, rewards: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
Accuracy: 1.0
Model: RealisticVision


You have disabled the safety checker for <class 'diffusers.pipelines.stable_diffusion.pipeline_stable_diffusion.StableDiffusionPipeline'> by passing `safety_checker=None`. Ensure that you abide to the conditions of the Stable Diffusion license and do not expose unfiltered results in services or applications open to the public. Both the diffusers team and Hugging Face strongly recommend to keep the safety filter enabled in all public facing circumstances, disabling it only for use-cases that involve analyzing network behavior or auditing its results. For more information, please have a look at https://github.com/huggingface/diffusers/pull/254 .
100%|██████████| 30/30 [00:00<00:00, 39.43it/s]


Model: RealisticVision, rewards: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
Accuracy: 1.0


## Check if miner can cheat by running less steps than others

In [9]:
import requests
import json
import torch
from dependency_modules.rewarding.app import instantiate_from_config, MODEL_CONFIG
from image_generation_subnet.validator.forward import get_reward
from image_generation_subnet.protocol import ImageGenerating
from dependency_modules.rewarding.utils import pil_image_to_base64
import numpy as np

N = 20
url_dict = {
    "RealisticVision": "http://check_realistic_vision_nicheimage.nichetensor.com:15011/verify",
    "SDXLTurbo": "http://sdxl_turbo_nicheimage.nichetensor.com:15012/verify",
}
rewards = {}
for model, url in url_dict.items():
    print(f"Model: {model}")
    pipe = instantiate_from_config(MODEL_CONFIG[model])
    rewards[model] = []
    if model == "RealisticVision":
        params = {
            "num_inference_steps": 30,
        }
    else:
        params = {
            "num_inference_steps": 4,
            "width": 512,
            "height": 512,
            "guidance_scale": 0.5,
        }
    for i in range(N):
        prompt = "an image of samurai, red hat, and a katana, with a red background"
        seed = i
        synapse = ImageGenerating(
            prompt=prompt,
            seed=seed,
            model_name=model,
            pipeline_params=params,
        )
        generator = torch.manual_seed(seed)
        params["num_inference_steps"] = 1 if model == "SDXLTurbo" else 10
        result = pipe(
            prompt,
            generator=generator,
            **params,
        )
        params["num_inference_steps"] = 4 if model == "SDXLTurbo" else 30
        images = result.images
        synapse.image = pil_image_to_base64(images[0])
        reward = get_reward(url, [synapse], synapse)[0]
        print(reward)
        rewards[model].append(reward)

    print(f"Model: {model}, rewards: {rewards[model]}")
    print(f"Accuracy: {np.mean(rewards[model])}")

Model: RealisticVision


You have disabled the safety checker for <class 'diffusers.pipelines.stable_diffusion.pipeline_stable_diffusion.StableDiffusionPipeline'> by passing `safety_checker=None`. Ensure that you abide to the conditions of the Stable Diffusion license and do not expose unfiltered results in services or applications open to the public. Both the diffusers team and Hugging Face strongly recommend to keep the safety filter enabled in all public facing circumstances, disabling it only for use-cases that involve analyzing network behavior or auditing its results. For more information, please have a look at https://github.com/huggingface/diffusers/pull/254 .
100%|██████████| 10/10 [00:00<00:00, 38.81it/s]


0


100%|██████████| 10/10 [00:00<00:00, 38.82it/s]


1


...

0


100%|██████████| 1/1 [00:00<00:00, 15.72it/s]


0


100%|██████████| 1/1 [00:00<00:00, 15.83it/s]


0
Model: SDXLTurbo, rewards: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Accuracy: 0.0
